# DenseNet Model Implementation

In [1]:
import os
import glob
import cv2
import math
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import Subset
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
#import sklearn
#from PIL import Image
import zipfile
#import urllib.request
import os.path
#from IPython.display import display
import dotenv
dotenv.load_dotenv()

True

# Set GPU access

In [2]:
if torch.backends.mps.is_available(): # Check if PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
    print(f"MPS is available!")
    if torch.backends.mps.is_built():
        print(f"MPS (Metal Performance Shader) is built in!")    
    device = "mps"
elif torch.cuda.is_available():
    print(f"CUDA is available!")
    device = "cuda"
else:
    print(f"Only CPU is available!")
    device = "cpu"
print(f"Using device: {device}")

CUDA is available!
Using device: cuda


# Helper functions

In [3]:
def create_image_stack(images, labels, nImLevelsData):
    
    image_count = len(images) // nImLevelsData  

    stacked_data = []
    stacked_labels = []

    normalize = transforms.Normalize([0.485], [0.229]) 

    for i in range(image_count):
        start_index = i * nImLevelsData
        end_index = start_index + nImLevelsData

        single_stack = images[start_index+1:end_index] # only 8 images without background image

        image_stack = np.stack([normalize(torch.from_numpy(img.astype(np.float32))) for img in single_stack])
        image_stack = torch.from_numpy(image_stack)
        
        stacked_data.append(image_stack)
        stacked_labels.append(labels[start_index])
    
    return stacked_data, stacked_labels

In [4]:
def compute_score_with_logits(logits, labels):
    logitsM = torch.max(logits, 1)[1].data # argmax
    if device == "mps":
        one_hots = torch.zeros(*labels.size()).to(device)
    elif device == "cuda":
        one_hots = torch.zeros(*labels.size()).cuda()
    else:
        one_hots = torch.zeros(*labels.size())
    one_hots.scatter_(1, logitsM.view(-1, 1), 1)
    scores = (one_hots * labels)

    return scores

In [5]:
def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
    return torch.index_select(a, dim, order_index)

In [6]:
def create_classEncoding(classes):
    Encoding = {}
    for i,class_ in enumerate(classes):
        
        tensor_values = [0] * len(classes)
        tensor_values[i] = 1
        Encoding[class_] = torch.FloatTensor(tensor_values)

    return Encoding

# Preprocessing

In [7]:
class DataPreprocessing(Dataset):
    
    # classPaths --> className: [path, #training_samples]
    imagesFromEachClass = eval(os.getenv("imagesFromEachClass"))

    classPaths = {
    
    'AGE_RMD': ['../../Images/CT_RETINA/AGE_RMD_55/AR1-45_9Levels/', min(imagesFromEachClass, 45)],
    'CSR':     ['../../Images/CT_RETINA/CSR_102/CR1-80_9Levels/', min(imagesFromEachClass,80)],
    'DIABETR': [ '../../Images/CT_RETINA/DIABETR_107/DR1-83_9Levels/', min(imagesFromEachClass,83)],
    'MACHOLE': ['../../Images/CT_RETINA/MACHOLE_102/MH1-80_9Levels/', min(imagesFromEachClass,80)],
    'NORMAL':  ['../../Images/CT_RETINA/NORMAL_206/NR1-160_9Levels/', min(imagesFromEachClass,160)] 
        }
    
    def __init__(self, classPaths=classPaths):
        
        self.image_paths = []
        self.labels = []
        self.images = []

        self.classes = eval(os.getenv("CLASSES"))
        self.classEncoding = create_classEncoding(self.classes)
        self.nImLevelsData = eval(os.getenv("nImLevelsData"))
        self.downImageSize = eval(os.getenv("downImageSize"))
    
        # image paths
        for imCoreName in (self.classEncoding.keys()):
            temp_paths = []
            for directoryPath in glob.glob(classPaths[imCoreName][0]):
                for imgPath in glob.glob(os.path.join(directoryPath, "*.jpg")):
                    temp_paths.append(imgPath)

            # labels 
            labels_list = [imCoreName] * classPaths[imCoreName][1] * self.nImLevelsData

            for label in labels_list:
                labelTensor = torch.FloatTensor(np.zeros(len(self.classes)))

                labelTensor = labelTensor.add(self.classEncoding[label])
                self.labels.append(labelTensor)

            img_paths = temp_paths[:classPaths[imCoreName][1] * self.nImLevelsData] 
            self.image_paths.append(img_paths)

            for image_path in img_paths:
                img = cv2.imread(image_path,0) 
                img = cv2.resize(img, (self.downImageSize, self.downImageSize), interpolation = cv2.INTER_AREA)
                img = np.reshape(img, (*img.shape, 1))
                img = np.transpose(img, (2, 0, 1))
                self.images.append(img)
              
        self.image_paths = [y for x in self.image_paths for y in x]
        self.stacked_images, self.stacked_labels = create_image_stack(self.images, self.labels, self.nImLevelsData)  
                
    def __getitem__(self, index):
        # preprocess and return single image stack of dim 8*1*224*224
        return self.stacked_images[index], self.stacked_labels[index]
    
    def __len__(self):

        return len(self.stacked_images)

# Model

In [8]:
class DenseNet121(nn.Module):
    def __init__(self):
        super(DenseNet121, self).__init__()

        self.classes = eval(os.getenv("CLASSES"))
        self.model = torchvision.models.densenet121(pretrained = True)

        num_ftrs = self.model.classifier.in_features
        self.model.features[0] = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, len(self.classes)),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [9]:
data = DataPreprocessing()

testOnly = eval(os.getenv("testOnly"))
if testOnly:
    trainTestSplit = 0
else:
    trainTestSplit = eval(os.getenv("trainTestSplit"))

trainShuffleFlag = eval(os.getenv("trainShuffleFlag"))
seqSplit = eval(os.getenv("seqSplit"))

trainSize = math.ceil(len(data) * trainTestSplit)
testSize = math.floor(round(len(data) * (1-trainTestSplit)))

if seqSplit:
    nClasses = eval(os.getenv("nClasses"))
    trainSizePerClass = trainSize / nClasses
    testSizePerClass = testSize / nClasses
    imPerClass = trainSizePerClass + testSizePerClass

    trainTokens = []
    testTokens  = []

    for i in range(nClasses):
        trainTokens = np.append(trainTokens, np.arange(imPerClass*i,trainSizePerClass*(i+1)))
        testTokens = np.append(testTokens, np.arange(imPerClass*i + trainSizePerClass ,imPerClass*(i+1))) 

    trainTokens = trainTokens.astype(np.int64) 
    testTokens = testTokens.astype(np.int64) 

    train_set = Subset(data, trainTokens)
    test_set = Subset(data, testTokens)
else:
    train_set, test_set = random_split(data, [trainSize, testSize])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=2, shuffle=trainShuffleFlag, num_workers=0)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)

if device == "mps":
    model = DenseNet121().to(device)
    model = nn.DataParallel(model).to(device)
elif device == "cuda":
    model = DenseNet121().cuda()
    model = nn.DataParallel(model).cuda()
else :
    model = DenseNet121()
    model = nn.DataParallel(model)


c:\Users\Temp\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Temp\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:

%%time

#parameters to save/load models
testOnly = os.getenv('testOnly')
if testOnly:
    saveModel = 'False'
else:
    saveModel = os.getenv('saveModel')

modelPath = os.getenv('SAVE_LOAD_MODEL_PATH')
downImageSize = os.getenv('downImageSize')
nClasses = os.getenv('nClasses')
modelFname = "dense" + str(nClasses) + "class" + str(downImageSize) + "pix.pt"
modelPathFile = modelPath + modelFname

if testOnly=='False': #We are doing full training 

    # HERE WE NEED TO BE ABLE TO INSERT INITIAL GUESS FROM PREVIOUSLY TRAINED MODEL

    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # RMSprop, Adam

    nEpochs = eval(os.getenv("nEpochs"))
    useImLevels = eval(os.getenv("useImLevels"))

    for epoch in range(nEpochs): #loop over the dataset multiple times

        running_loss = 0.0
        correct = 0
        total = 0 
        for i, (images, labels) in enumerate(trainloader, 0): # get the inputs; data is a list of [images, labels]

            # zero the parameter gradients
            optimizer.zero_grad()
            
            if device == "mps":
                images = images.to(device)
                labels = tile(labels, 0, useImLevels).to(device) #duplicate for each crop the label 
            elif device == "cuda":
                images = images.cuda()
                labels = tile(labels, 0, useImLevels).cuda()
            else:
                labels = tile(labels, 0, useImLevels)
        
            #format input
            n_batches, n_crops, channels, height, width = images.size()
            image_batch = torch.autograd.Variable(images.view(-1, channels, height, width)) 
            
            # forward + backward + optimize
            outputs = model(image_batch)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            correct += compute_score_with_logits(outputs, labels).sum()
            total += labels.size(0)

        print('Epoch: %d, loss: %.3f, Accuracy: %.3f' %
            (epoch + 1, running_loss, 100 * correct / total))

    print('Finished Training')
    if saveModel: 
        torch.save(model, modelPathFile)
        
#load the model when doing only testing
else: 
    model = torch.load(modelPathFile)


Epoch: 1, loss: 28.974, Accuracy: 26.953
Epoch: 2, loss: 27.777, Accuracy: 30.339
Epoch: 3, loss: 26.565, Accuracy: 33.854
Epoch: 4, loss: 25.169, Accuracy: 40.234
Epoch: 5, loss: 23.881, Accuracy: 45.833
Epoch: 6, loss: 22.641, Accuracy: 51.432
Epoch: 7, loss: 21.471, Accuracy: 55.208
Epoch: 8, loss: 20.300, Accuracy: 61.719
Epoch: 9, loss: 19.701, Accuracy: 64.193
Epoch: 10, loss: 18.728, Accuracy: 67.839
Epoch: 11, loss: 17.661, Accuracy: 72.005
Epoch: 12, loss: 16.541, Accuracy: 75.391
Epoch: 13, loss: 15.216, Accuracy: 79.427
Epoch: 14, loss: 14.246, Accuracy: 83.464
Epoch: 15, loss: 13.385, Accuracy: 85.026
Epoch: 16, loss: 12.528, Accuracy: 85.547
Epoch: 17, loss: 11.629, Accuracy: 86.719
Epoch: 18, loss: 10.900, Accuracy: 89.583
Epoch: 19, loss: 10.259, Accuracy: 91.146
Epoch: 20, loss: 9.602, Accuracy: 93.099
Epoch: 21, loss: 9.066, Accuracy: 92.578
Epoch: 22, loss: 8.681, Accuracy: 95.052
Epoch: 23, loss: 8.243, Accuracy: 95.573
Epoch: 24, loss: 7.804, Accuracy: 95.833
Epoch:

In [11]:
#model evaluation
model.eval()

DataParallel(
  (module): DenseNet121(
    (model): DenseNet(
      (features): Sequential(
        (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu0): ReLU(inplace=True)
        (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (denseblock1): _DenseBlock(
          (denselayer1): _DenseLayer(
            (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace=True)
            (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace=True)
            (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (denselayer2): _DenseLayer

In [12]:
#if useCalcModel:
#    model = torch.load('model.pth')
correct = 0
total = 0
useImLevels = eval(os.getenv("useImLevels"))
perfect8 = 0

with torch.no_grad():
    for i, (images, labels) in enumerate(testloader, 0):
        if device == "mps":
            images = images.to(device)
            labels = tile(labels, 0, useImLevels).to(device)
        elif device == "cuda":
            images = images.cuda()
            labels = tile(labels, 0, useImLevels).cuda()
        else:
            labels = tile(labels, 0, useImLevels)
        n_batches, n_crops, channels, height, width = images.size()
        image_batch = torch.autograd.Variable(images.view(-1, channels, height, width))
      
        outputs = model(image_batch)
        fitScore = compute_score_with_logits(outputs, labels).sum()
        correct += fitScore.item()
        total += labels.size(0)
        print(i+1, fitScore.item())
        if (fitScore.item() == 8):
            perfect8 +=1 

    
print('Accuracy on test set: %.3f' % (100 * correct / total))
print('Images with perfect 8: '+ str(perfect8))

1 4.0
2 7.0
3 8.0
4 1.0
5 7.0
6 8.0
7 4.0
8 0.0
9 6.0
10 3.0
11 2.0
12 1.0
13 8.0
14 8.0
15 4.0
16 7.0
17 8.0
18 8.0
19 8.0
20 8.0
21 8.0
22 8.0
23 8.0
24 7.0
Accuracy on test set: 73.438
Images with perfect 8: 11
